In [1]:
#import helper
import pandas as pd
#from utils import *

import time
import numpy as np
import math
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn

from sklearn.decomposition import PCA

from IPython.display import Image

import pennylane as qml


# Introduction

Stock price prediction is one of the most rewarding problems in modern finance, where the accurate forecasting of future stock prices can yield significant profit and reduce the risks. LSTM (Long Short-Term Memory) is a recurrent Neural Network (RNN) applicable to a broad range of problems aiming to analyze or classify sequential data. Therefore, many people have used LSTM to predict the future stock price based on the historical data sequences with great success.

On the other hand, recent studies have shown that the LSTM's efficiency and trainability can be improved by replacing some of the layers in the LSTM with variational quantum layers, thus making it a quantum-classical hybrid model of LSTM which we will call QLSTM for Quantum LSTM. In the study done by Samuel Yen-Chi Chen, Shinjae Yoo, and Yao-Lung L. Fang, they show that QLSTM offers better trainability compared to its classical counterpart as it proved to learn significantly more information after the first training epoch than its classical counterpart, learnt the local features better, all while having a comparable number of parameters. Inspired by these recent results, we proceed to test this variational quantum-classical hybrid neural network technique on stock price predictions. 

In the following notebook, we show a proof of concept that QLSTM can be used to great effect for stock price prediction, offering comparable and arguably better results than its classical counterpart. To do so, we implement both LSTM and QLSTM to predict the stock prices of the company Merck & Co. Inc (MRK) with the same number of features, of which we chose based on earlier studies done with stock price predictions.

This submission was motivated by a combination of a few separate studies:

Stock price prediction use BERT and GAN: https://arxiv.org/pdf/2107.09055.pdf, Priyank Sonkiya, Vikas Bajpai, Anukriti Bansal <br>
Quantum Long Short-Term Memory: https://arxiv.org/pdf/2009.01783.pdf, Samuel Yen-Chi Chen, Shinjae Yoo, and Yao-Lung L. Fang <br>

With code and ideas reused and repurposed from the following sources:

Example of a QLSTM: https://github.com/rdisipio/qlstm, Riccardo Di Sipio <br>
How to use PyTorch LSTMs for time series regression: https://www.crosstab.io/articles/time-series-pytorch-lstm, Brian Kent <br>
Using GANs to predict stock price movement: https://towardsdatascience.com/aifortrading-2edd6fac689d, Boris Banushev<br>

## Brief Outline

To demonstrate the use of QLSTM for stock prediction, we use the stock prices of the company Merck & Co. Inc (MRK). The notebook will proceed in the following manner:

1. Brief description of Data
2. Using Classical LSTM to perform stock price prediction
3. Defining QLSTM and using it to perform stock price prediction
4. Comparison between LSTM and QLSTM

Note that for the LSTM, we would be using PyTorch; while for the QLSTM we would be using PyTorch and Pennylane. 

# Experimentation

- weight decay coefficient (weight decay): 0, 0.0001, 0.01 
- width of the layer (width)(controlled by hidden units): 16, 128, 512 
- mini-batch size (batch size): 1, 5, 10
- learning rate (learning rate):  0.001, 0.05, 0.1
- dropout probability (dropout)(only classical for now): 0, 0.05, 0.3
- depth of the architecture(depth): 1, 6, 12
- sequence length: 3, 7, 30
- num_epochs: 5, 10, 20

number of iterations = batch_size * num_epochs

In [2]:
batch_size = 1
sequence_length = 30
dropout_value = 0
num_layers = 1

weight_decay_value = 0
learning_rate = 0.05

num_hidden_units = 16
num_epochs = 20

- No. of Qubits: 4, 8, 12
- Feature Map: RY + RZ, 
- Entanglement structure: Circular, Full
- Rotation structure: Full 3
- Number of layers: 1


In [3]:
class QLSTM(nn.Module):
    def __init__(self, 
                input_size, 
                hidden_size, 
                n_qubits=4, # No. of Qubits:
                n_qlayers=1, # Number of layers:
                n_vrotations=3,
                batch_first=True,
                return_sequences=False, 
                return_state=False,
                backend="default.qubit"):
        super(QLSTM, self).__init__()
        self.n_inputs = input_size
        self.hidden_size = hidden_size
        self.concat_size = self.n_inputs + self.hidden_size
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.n_vrotations = n_vrotations
        self.backend = backend  # "default.qubit", "qiskit.basicaer", "qiskit.ibm"

        self.batch_first = batch_first
        self.return_sequences = return_sequences
        self.return_state = return_state
        
        self.wires_forget = [f"wire_forget_{i}" for i in range(self.n_qubits)]
        self.wires_input = [f"wire_input_{i}" for i in range(self.n_qubits)]
        self.wires_update = [f"wire_update_{i}" for i in range(self.n_qubits)]
        self.wires_output = [f"wire_output_{i}" for i in range(self.n_qubits)]

        self.dev_forget = qml.device(self.backend, wires=self.wires_forget)
        self.dev_input = qml.device(self.backend, wires=self.wires_input)
        self.dev_update = qml.device(self.backend, wires=self.wires_update)
        self.dev_output = qml.device(self.backend, wires=self.wires_output)

        #self.dev_forget = qml.device(self.backend, wires=self.n_qubits)
        #self.dev_input = qml.device(self.backend, wires=self.n_qubits)
        #self.dev_update = qml.device(self.backend, wires=self.n_qubits)
        #self.dev_output = qml.device(self.backend, wires=self.n_qubits)
        
        def ansatz(params, wires_type):
            # Entangling layer: Circular
            for j in range(self.n_qubits):
                if j < (self.n_qubits - 1):
                    qml.CNOT(wires=[wires_type[j], wires_type[j + 1]])
                else:
                    qml.CNOT(wires=[wires_type[j], wires_type[0]])

            # Variational layer: Full 3
            for i in range(self.n_qubits):
                qml.RX(params[0][i], wires=wires_type[i])
                qml.RY(params[1][i], wires=wires_type[i])
                qml.RZ(params[2][i], wires=wires_type[i])
                
        def VQC(features, weights, wires_type):
            # Preproccess input data to encode the initial state.
            #qml.templates.AngleEmbedding(features, wires=wires_type)
            # Feature Map
            ry_params = [torch.arctan(feature) for feature in features]
            rz_params = [torch.arctan(feature**2) for feature in features]
            for i in range(self.n_qubits):
                qml.Hadamard(wires=wires_type[i])
                qml.RY(ry_params[i], wires=wires_type[i])
                qml.RZ(ry_params[i], wires=wires_type[i])
        
            #Variational block.
            qml.layer(ansatz, self.n_qlayers, weights, wires_type = wires_type)

        def _circuit_forget(inputs, weights):
            VQC(inputs, weights, self.wires_forget)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_forget]
        self.qlayer_forget = qml.QNode(_circuit_forget, self.dev_forget, interface="torch")

        def _circuit_input(inputs, weights):
            VQC(inputs, weights, self.wires_input)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_input]
        self.qlayer_input = qml.QNode(_circuit_input, self.dev_input, interface="torch")

        def _circuit_update(inputs, weights):
            VQC(inputs, weights, self.wires_update)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_update]
        self.qlayer_update = qml.QNode(_circuit_update, self.dev_update, interface="torch")

        def _circuit_output(inputs, weights):
            VQC(inputs, weights, self.wires_output)
            return [qml.expval(qml.PauliZ(wires=i)) for i in self.wires_output]
        self.qlayer_output = qml.QNode(_circuit_output, self.dev_output, interface="torch")

        weight_shapes = {"weights": (self.n_qlayers, self.n_vrotations, self.n_qubits)}
        print(f"weight_shapes = (n_qlayers, n_vrotations, n_qubits) = ({self.n_qlayers}, {self.n_vrotations}, {self.n_qubits})")

        self.clayer_in = torch.nn.Linear(self.concat_size, self.n_qubits)
        self.VQC = {
            'forget': qml.qnn.TorchLayer(self.qlayer_forget, weight_shapes),
            'input': qml.qnn.TorchLayer(self.qlayer_input, weight_shapes),
            'update': qml.qnn.TorchLayer(self.qlayer_update, weight_shapes),
            'output': qml.qnn.TorchLayer(self.qlayer_output, weight_shapes)
        }
        self.clayer_out = torch.nn.Linear(self.n_qubits, self.hidden_size)
        #self.clayer_out = [torch.nn.Linear(n_qubits, self.hidden_size) for _ in range(4)]

    def forward(self, x, init_states=None):
        '''
        x.shape is (batch_size, seq_length, feature_size)
        recurrent_activation -> sigmoid
        activation -> tanh
        '''
        if self.batch_first is True:
            batch_size, seq_length, features_size = x.size()
        else:
            seq_length, batch_size, features_size = x.size()

        hidden_seq = []
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size)  # hidden state (output)
            c_t = torch.zeros(batch_size, self.hidden_size)  # cell state
        else:
            # for now we ignore the fact that in PyTorch you can stack multiple RNNs
            # so we take only the first elements of the init_states tuple init_states[0][0], init_states[1][0]
            h_t, c_t = init_states
            h_t = h_t[0]
            c_t = c_t[0]

        for t in range(seq_length):
            # get features from the t-th element in seq, for all entries in the batch
            if self.batch_first is True:
                x_t = x[:, t , :]
            else:
                x_t = x[t, : , :]
            
            # Concatenate input and hidden state
            v_t = torch.cat((h_t, x_t), dim=1)

            # match qubit dimension
            y_t = self.clayer_in(v_t)

            f_t = torch.sigmoid(self.clayer_out(self.VQC['forget'](y_t)))  # forget block
            i_t = torch.sigmoid(self.clayer_out(self.VQC['input'](y_t)))  # input block
            g_t = torch.tanh(self.clayer_out(self.VQC['update'](y_t)))  # update block
            o_t = torch.sigmoid(self.clayer_out(self.VQC['output'](y_t))) # output block

            c_t = (f_t * c_t) + (i_t * g_t)
            h_t = o_t * torch.tanh(c_t)

            hidden_seq.append(h_t.unsqueeze(0))
        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)

In [4]:
from torch.utils.tensorboard import SummaryWriter

In [5]:
writer = SummaryWriter()

# Data

To briefly describe the data, we have collected the historical data of the MRK stock prices, of which we focus on the closing price. Our goal is then to forecast the closing stock prices of MRK using LSTM (or QLSTM). 

To achieve this goal, we have collected the data and information we thought necessary and important. This includes the following:

- Technical indicators
- Trend approximations (Fourier Transforms)
- ARIMA 
- Correlated assets
- Sentimental analysis

While interesting and important in its own right, we have decided not to go into detail for the data collection in this notebook. For more information, please go take a look at the Data Collection notebook also in this Github. 

In this section, our goal is to process the data to get it ready for the LSTM and QLSTM.

First, we read in the data, dropping the index and the date.

In [6]:
df = pd.read_csv('dataset_MRK_prediction.csv')
df = df.drop(['Date' ,'Unnamed: 0'], axis=1)
df

,Close,Volume,ma7,ma21,26ema,12ema,MACD,20sd,upper_band,lower_band,...,GSK,LLY,NVS,NYSE,NASDAQ,FT3,FT6,FT9,ARIMA,Close_lead1
0,31.011450,14227229,30.050436,29.200291,29.514006,29.867322,0.353316,0.835852,30.871996,27.528586,...,20.441219,22.546120,26.214659,6671.140137,2017.979980,55.454947,50.337802,47.947327,31.011450,31.440840
1,31.440840,20081566,30.318975,29.299346,29.688906,30.115697,0.426791,0.963549,31.226444,27.372247,...,20.290920,22.639698,26.532934,6697.220215,2024.229980,55.359252,50.140845,47.659130,31.440840,31.183207
2,31.183207,10438080,30.564340,29.420211,29.822316,30.283528,0.461212,1.026446,31.473102,27.367319,...,20.280550,22.492653,26.608170,6687.939941,2024.430054,55.263482,49.944470,47.372320,31.183207,31.364504
3,31.364504,10302154,30.858779,29.556525,29.957940,30.452906,0.494965,1.086420,31.729366,27.383684,...,20.513784,22.800129,26.984316,6722.310059,2027.729980,55.167638,49.748691,47.086936,31.364504,30.839695
4,30.839695,12640452,30.973283,29.664667,30.034423,30.513340,0.478916,1.094567,31.853801,27.475534,...,20.285738,22.459227,26.857002,6709.040039,2028.770020,55.071723,49.553518,46.803017,30.839695,30.944656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,78.301529,13675457,75.931026,74.702836,75.305196,75.725497,0.420300,1.705192,78.113220,71.292452,...,38.856968,148.141144,82.393753,12302.190430,9682.910156,57.910147,55.622779,55.790021,76.887436,77.814888
2713,77.814888,9292930,76.515814,74.863687,75.491099,76.046941,0.555842,1.816324,78.496336,71.231038,...,38.423382,146.373550,81.836136,12286.980469,9615.809570,57.817099,55.415062,55.480641,78.191500,78.492363
2714,78.492363,10413347,77.157852,75.097238,75.713415,76.423160,0.709745,1.857931,78.813100,71.381377,...,38.469505,144.916718,82.576424,12641.440430,9814.080078,57.723926,55.207594,55.171523,77.972240,79.103050
2715,79.103050,9956838,77.684024,75.428935,75.964499,76.835451,0.870952,1.960788,79.350511,71.507360,...,39.087601,144.916718,82.364914,12836.599609,9924.750000,57.630627,55.000389,54.862711,78.309688,78.492363


We identify the dependent and independent variables:

In [7]:
target = "Close_lead1"

In [8]:
features = list(df.columns.difference(["Close", 'Close_lead1']))
features

['12ema',
 '20sd',
 '26ema',
 'AMGN',
 'ARIMA',
 'BMY',
 'FT3',
 'FT6',
 'FT9',
 'GSK',
 'JNJ',
 'LLY',
 'MACD',
 'NASDAQ',
 'NVS',
 'NYSE',
 'PFE',
 'SNP',
 'SNY',
 'VTRS',
 'VZ',
 'Volume',
 'ema',
 'log_momentum',
 'lower_band',
 'ma21',
 'ma7',
 'momentum',
 'neg',
 'neu',
 'pos',
 'upper_band']

## Data Processing

To process the data, we first split it into training and test data, where two-thirds of the data is used for training, and the last third is used for testing.

In [9]:
size = int(len(df) * 0.67)

df_train = df.loc[:size].copy()
df_test = df.loc[size:].copy()

Next, in order to ensure that some values don't inherently dominate the features due to their magnitude, we standardize their values.

In [10]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

In [11]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[self.target].values).float()
        self.X = torch.tensor(dataframe[self.features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

Finally, the last step in the data processing to prepare it for LSTM is to prepare the data in a sequence of past observations. Preparing LSTM on time series data means that it uses a certain number of past observations to predict the future. In this case, the sequence length decides how many days the LSTM considers in advance. If the sequence length is $n$, then the LSTM considers the last $n$ observations to predict the $n+1$th day.

The sequence length we decided on is 3 for purposes of this notebook.

In [12]:
torch.manual_seed(101)

#batch_size = 10
#sequence_length = 3

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([1, 30, 32])
Target shape: torch.Size([1])


In [13]:
i = 0
for _ in train_loader:
    i += 1 
print(i)

1821


# Classical LSTM

We first define two functions:
    
- train_model: function to train the model based on the batches of data
- test_model: function to test the model on the testing data
    
For both, they print the loss at the end to let us know how the model is performing with regards to the data.

In [14]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")
    return avg_loss

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")
    return avg_loss

## Running the Classical LSTM

To understand how we implemented QLSTM, we have to first understand how we implemented LSTM. LSTM follows the following structure:

<img src="lstm2.jpg" alt="drawing" width="400"/>

Image taken from: Quantum Long Short-Term Memory, https://arxiv.org/pdf/2009.01783.pdf. By Samuel Yen-Chi Chen, Shinjae Yoo, and Yao-Lung L. Fang

Simply put, LSTM uses 4 neural network layers in each LSTM cell to perform its functions. They are:

- Forget layer
- Input layer
- Update layer
- Output layer

You can see the corresponding layers in the W cells in the picture above. We will be skipping the technical details, as you can find better information on the theory elsewhere, but it is important to note that these 4 layers are the key to building an LSTM neural network model that we can train and eventually use to predict, and usually take the form of a normal NN layer (like a linear layer with reLU or convolutional layers).

LSTMs are well studied, and there is a native implementation of it in PyTorch to begin with, so we use a slightly modified version of it for the time series LSTM that we perform here. The code for the time series LSTM was reused from:

How to use PyTorch LSTMs for time series regression: https://www.crosstab.io/articles/time-series-pytorch-lstm, Brian Kent.

In the following code, we train LSTM to predict future stock prices, and then test it on the test dataset. The learning rate of 0.0001 was decided after some experimentation, where we chose the learning rate that most reliably gave accurate results. The number of epochs we use is 20, by which it would have converged and thus would suffice for the purposes of this notebook. After that, we proceed to see 3 different graphs: the comparison between the real stock prices and the ones given by the model; and the evolution of test loss and training loss by epoch.

# Experimentation

In [16]:
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units, n_qubits=0, n_qlayers=1
                 #, num_layers, dropout_value
                ):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout_value = dropout_value

        self.lstm = QLSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            n_qubits = n_qubits,
            n_qlayers= n_qlayers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [17]:
#learning_rate = 0.0001
#num_hidden_units = 16

model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units, n_qubits=4)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= weight_decay_value)

weight_shapes = (n_qlayers, n_vrotations, n_qubits) = (1, 3, 4)


In [ ]:
classical_loss_train = []
classical_loss_test = []
classical_time = []
print("Untrained test\n--------")
start = time.time()
test_loss = test_model(test_loader, model, loss_function)
end = time.time()
print("Execution time", end - start)
classical_loss_test.append(test_loss)

for ix_epoch in range(num_epochs):
    print(f"Epoch {ix_epoch}\n---------")
    start = time.time()
    train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_loss = test_model(test_loader, model, loss_function)
    end = time.time()
    print("Execution time", end - start)
    classical_loss_train.append(train_loss)
    classical_loss_test.append(test_loss)
    classical_time.append(end - start)

Untrained test
--------
Test loss: 6.597873733195174
Execution time 1302.7725920677185
Epoch 0
---------


C:\Users\jonas\Anaconda3\envs\qrnn\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Train loss: 0.14816461981785436
Test loss: 2.7046614232575106
Execution time 5318.334117650986
Epoch 1
---------
Train loss: 0.13623541970835903
Test loss: 2.7175407673535235
Execution time 5319.161185264587
Epoch 2
---------
Train loss: 0.14224370016102994
Test loss: 3.5212133973020077
Execution time 5315.983252048492
Epoch 3
---------
Train loss: 0.09548015852759979
Test loss: 2.594655237563075
Execution time 5398.879703998566
Epoch 4
---------
Train loss: 0.09206215778050478
Test loss: 3.008570600040512
Execution time 5489.611770868301
Epoch 5
---------
Train loss: 0.10468765771423166
Test loss: 2.78595284589459
Execution time 5503.001051664352
Epoch 6
---------
Train loss: 0.09625351373622787
Test loss: 3.03025396478397
Execution time 5326.096094608307
Epoch 7
---------
Train loss: 0.10390485516926543
Test loss: 2.687387861979946
Execution time 5325.742523431778
Epoch 8
---------
Train loss: 0.09396903098013216
Test loss: 3.2179606967904673
Execution time 5308.123530864716
Epoch 9


We then use the model to predict the test set, and then compare the results of the prediction to the real values.

In [ ]:
def predict(data_loader, model):
    """Just like `test_loop` function but keep track of the outputs instead of the loss
    function.
    """
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output

In [ ]:
train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_train[ystar_col] = predict(train_eval_loader, model).numpy()
df_test[ystar_col] = predict(test_loader, model).numpy()

df_out = pd.concat((df_train, df_test))[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

print(df_out)

In [ ]:
df_out = df_out[~df_out.index.duplicated(keep='first')]

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(range(len(df_out)), df_out["Close_lead1"], label = "Real")
plt.plot(range(len(df_out)), df_out["Model forecast"], label = "LSTM Prediction")
plt.ylabel('Stock Price')
plt.xlabel('Days')
plt.vlines(size, ymin = 30, ymax = 90, label = "Test set start", linestyles = "dashed")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(21), classical_loss_test)
plt.ylabel('Test Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, 21), classical_loss_train)
plt.ylabel('Train Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
avg_time = sum(classical_time)/num_epochs
avg_time

## Number of parameters used

In [ ]:
def count_parameters(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

In [ ]:
num_par = count_parameters(model)
num_par

# Trading Strategy

In [ ]:
def trading_model(data_loader, model, loss_function, initial_inv, initial_price):
    
    # initial seed money
    inv = initial_inv
    
    # initial price of stock
    price = initial_price
    
    # initial number of stock 
    number_stocks = 0
    
    # predicted earnings
    pred_earn = 0 
    
    # predicted stock
    pred_output = []
    
    # day
    day = 1
    
    for X, y in data_loader:
        model.eval()
        with torch.no_grad():
            output = model(X)
            
            pred_price =  output * target_stdev + target_mean
            pred_output.append(pred_price)
            print("Today's stock price: ", price)
            print("Predicted stock price for tomorrow: ", pred_price)
            
            #if predicted value next day higher than today's price
            if pred_price > price:
                number_stocks = number_stocks + inv/price
                inv = 0
                print("Day", day, ": Bought/Keep ", number_stocks, "stocks at price ", price)
                pred_earn = pred_earn + number_stocks * (pred_price - price)
                print("Predicted earnings: ", pred_earn)
                
                
            #if predicted value next day lower than (or equal to)  today's price
            else:
                inv = inv + number_stocks * price
                print("Day", day, ": Sold ", number_stocks, "at price ", price)
                number_stocks = 0
            
            # Update price
            price = y * target_stdev + target_mean
            day += 1
            print("Current number of stocks: ", number_stocks)
            print("Current seed money:", inv)
        
        model.train()
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    total_earnings = number_stocks*price + inv - initial_inv
    print(f"Trading earnings: {total_earnings}")
    print(f"Predicted earnings: {pred_earn}")
    return total_earnings, pred_earn, pred_output

In [ ]:
initial_inv = 10000
initial_price = df_out["Close_lead1"][size - 1]

In [ ]:
total_earnings, pred_earnings, pred_output = trading_model(test_loader, model, loss_function, initial_inv, initial_price)

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(range(len(df_out)), df_out["Close_lead1"], label = "Real")
plt.plot(range(size, len(df_out)), pred_output, label = "LSTM Trading Strategy Prediction")
plt.ylabel('Stock Price')
plt.xlabel('Days')
plt.vlines(size, ymin = 30, ymax = 90, label = "Test set start", linestyles = "dashed")
plt.legend()
plt.show()

In [ ]:
total_earnings

In [ ]:
pred_earnings

# Store Values

In [ ]:
import csv

In [ ]:
with open('test.csv', 'a', newline='') as test_csv:
    test_csv_writer = csv.writer(test_csv)
    test_csv_writer.writerow(classical_loss_test)

In [ ]:
with open('train.csv', 'a', newline='') as train_csv:
    train_csv_writer = csv.writer(train_csv)
    train_csv_writer.writerow(classical_loss_train)

In [ ]:
with open('pred.csv', 'a', newline='') as pred_csv:
    pred_csv_writer = csv.writer(pred_csv)
    pred_csv_writer.writerow(df_out["Model forecast"])

In [ ]:
with open('time.csv', 'a', newline='') as time_csv:
    time_csv_writer = csv.writer(time_csv)
    time_csv_writer.writerow([avg_time])

In [ ]:
with open('parameters.csv', 'a', newline='') as parameters_csv:
    parameters_csv_writer = csv.writer(parameters_csv)
    parameters_csv_writer.writerow([num_par])

In [ ]:
with open('total_earnings.csv', 'a', newline='') as total_earnings_csv:
    total_earnings_csv_writer = csv.writer(total_earnings_csv)
    total_earnings_csv_writer.writerow([total_earnings])

In [ ]:
with open('pred_earnings.csv', 'a', newline='') as pred_earnings_csv:
    pred_earnings_csv_writer = csv.writer(pred_earnings_csv)
    pred_earnings_csv_writer.writerow([pred_earnings])

In [ ]:
with open('pred_output.csv', 'a', newline='') as pred_output_csv:
    pred_output_csv_writer = csv.writer(pred_output_csv)
    pred_output_csv_writer.writerow(pred_output)

In [ ]:
for i in range(len(classical_loss_test)):
    writer.add_scalar("Test loss",classical_loss_test[i], i)
    
for i in range(len(classical_loss_train)):
    writer.add_scalar("Train loss: ", classical_loss_train[i], i)
    
for i in range(len(df_out["Model forecast"])):
    writer.add_scalar("Predicted Value: ", df_out["Model forecast"][i], i)
    
for i in range(len(dpred_output)):
    writer.add_scalar("Predicted Value (Dynamic): ", pred_output, size + i)
    
writer.add_scalar("Avg Time: ", avg_time, 1)
writer.add_scalar("Number of parameters: ", num_par, 1)
writer.add_scalar("Total earnings: ", total_earnings, 1)
writer.add_scalar("Predicted earnings: ", pred_earnings, 1)
    
writer.close()

## Complexity analysis

As a last point of comparison, we want to discuss the complexity analysis for the QLSTM, as well as compare the number of parameters used for the LSTM and the QLSTM.

Given that we would want to eventually use the QLSTM on real quantum computers, it would be prudent to discuss its viability in terms of the number of gates and qubits used. First of all, we look at the number of qubits. As can be seen, in a barebones early version of QLSTM, we can afford to use any number of qubits, even a small number like 4 to run it. This is because we can sandwich it between two classical layers, where the classical layers help to convert the vector sizes to the correct amount, from the embedding of input features into the quantum circuit to the processing of the output measurements. This is very similar in idea to dressed quantum circuits, which was presented in:

Transfer learning in hybrid classical-quantum neural networks: https://arxiv.org/abs/1912.08278, Andrea Mari, Thomas R. Bromley, Josh Izaac, Maria Schuld, Nathan Killoran

In the foreseeable future in the NISQ era, we would likely be continuing to use the same technique, although the number of qubits can be increased to improve the model, making this technique useful in the NISQ era.

For the number of gates used per circuit, we take a look at how the overall depth would increase as number of qubits and number of layers increases. As seen from the circuit above, the overall depth is $3 + n_l * (2 * n_q + 3)$, where $n_l, n_q$ refers to the number of layers and number of qubits respectively. Thus, the overall depth would increase linearly in both number of qubits and number of layers. What this also means is that the depth is quite easy to control, and extremely viable for the NISQ era. In this particular proof of example, we show that even when the number of layers is 1 (the smallest it can be) and the number of qubits is a small number, we still get pretty good results. 


# Conclusion and Future Outlook

In this notebook, we have given the proof of concept that QLSTM can be used to great effect for stock price prediction, offering comparable results than its classical counterpart, while needed much fewer parameters to train and getting more information per epoch. All in all, this shows that this technique has a lot of potential in the financial world and more, as anything time series related can be trained using an LSTM and hence also a QLSTM. This is relevant not only to stock price predictions, but also for example company sales predictions or other key performance predictions. However, there are still many areas for improvement.